In [1]:
import numpy as np
import scipy.optimize as spop
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller 

In [2]:
stocks = ['JPM', 'AAPL', 'HDB', 'V.MX', 'SBIN.NS', 'MCD', 'GOOG', 'RS', 'MSFT', 'CIPLA.BO']
start = '2019-12-31'
end = '2022-08-20'
fee = 0.001
window = 252
t_threshold = -2.5
data = pd.DataFrame()
returns = pd.DataFrame()
for stock in stocks:
    prices = yf.download(stock, start, end)
    data[stock] = prices['Close']
    returns[stock] = np.append(data[stock][1:].reset_index(drop=True)/data[stock][:-1].reset_index(drop=True) - 1, 0)

In [ ]:
data

,JPM,AAPL,HDB,V.MX,SBIN.NS,MCD,GOOG,RS,MSFT,CIPLA.BO
Date,,,,,,,,,,
2019-12-30,138.630005,72.879997,62.939999,3543.000000,NaN,196.910004,66.806999,120.089996,157.589996,NaN
2019-12-31,139.399994,73.412498,63.369999,3543.000000,333.750000,197.610001,66.850998,119.760002,157.699997,478.450012
2020-01-02,141.089996,75.087502,63.700001,3591.000000,339.299988,200.789993,68.368500,119.250000,160.619995,473.700012
2020-01-03,138.339996,74.357498,62.009998,3600.000000,333.700012,200.080002,68.032997,118.809998,158.619995,470.250000
2020-01-06,138.229996,74.949997,60.910000,3577.500000,319.000000,202.330002,69.710503,118.519997,159.029999,466.750000
...,...,...,...,...,...,...,...,...,...,...
2022-08-15,122.459999,173.190002,65.760002,4260.000000,NaN,265.440002,122.879997,196.050003,293.470001,NaN
2022-08-16,123.629997,173.029999,64.940002,4366.919922,525.950012,266.290009,122.510002,197.429993,292.709991,1033.849976
2022-08-17,122.589996,174.550003,64.610001,4261.819824,528.150024,266.820007,120.320000,194.800003,291.320007,1025.500000


In [ ]:
returns

,JPM,AAPL,HDB,V.MX,SBIN.NS,MCD,GOOG,RS,MSFT,CIPLA.BO
0,0.005554,0.007307,0.006832,0.000000,NaN,0.003555,0.000659,-0.002748,0.000698,NaN
1,0.012123,0.022816,0.005208,0.013548,0.016629,0.016092,0.022700,-0.004259,0.018516,-0.009928
2,-0.019491,-0.009722,-0.026531,0.002506,-0.016504,-0.003536,-0.004907,-0.003690,-0.012452,-0.007283
3,-0.000795,0.007968,-0.017739,-0.006250,-0.044052,0.011246,0.024657,-0.002441,0.002585,-0.007443
4,-0.017001,-0.004703,-0.003612,-0.004612,-0.001881,0.001483,-0.000624,0.002784,-0.009118,0.004285
...,...,...,...,...,...,...,...,...,...,...
661,0.009554,-0.000924,-0.012470,0.025099,NaN,0.003202,-0.003011,0.007039,-0.002590,NaN
662,-0.008412,0.008785,-0.005082,-0.024067,0.004183,0.001990,-0.017876,-0.013321,-0.004749,-0.008077
663,-0.007749,-0.002292,0.000155,0.018344,0.007952,-0.000900,0.004488,0.001437,-0.003948,0.006436
664,-0.024745,-0.015102,-0.024296,-0.005760,-0.022542,-0.000150,-0.022671,-0.017429,-0.013854,-0.001696


In [ ]:
#initialising arrays
gross_returns = np.array([])
net_returns = np.array([])
t_s = np.array([])
for i in range(0, 10):
    stock1 = stocks[0]
    for j in range(0, i+1):
        stock2 = stocks[1]
    #moving through the sample
    for t in range(window, len(data)):
        #defining the unit root function: stock2 = a + b*stock1
        def unit_root(b):
            a = np.average(data[stock2][t-window:t] - b*data[stock1][t-window:t])
            fair_value = a + b*data[stock1][t-window:t]
            diff = np.array(fair_value - data[stock2][t-window:t])
            diff_diff = diff[1:] - diff[:-1]
            reg = sm.OLS(diff_diff, diff[:-1])
            res = reg.fit()
            return res.params[0]/res.bse[0]
        #optimising the cointegration equation parameters
        res1 = spop.minimize(unit_root, data[stock2][t]/data[stock1][t], method='Nelder-Mead')
        t_opt = res1.fun
        b_opt = float(res1.x)
        a_opt = np.average(data[stock2][t-window:t] - b_opt*data[stock1][t-window:t])
        #simulating trading
        fair_value = a_opt + b_opt*data[stock1][t]
        if t == window:
            old_signal = 0
        if t_opt > t_threshold:
            signal = 0
            gross_return = 0
        else:
            signal = np.sign(fair_value - data[stock2][t])
            gross_return = signal*returns[stock2][t] - signal*returns[stock1][t]
        fees = fee*abs(signal - old_signal)
        net_return = gross_return - fees
        gross_returns = np.append(gross_returns, gross_return)
        net_returns = np.append(net_returns, net_return)
        t_s = np.append(t_s, t_opt)
        #interface: reporting daily positions and realised returns
        print('day '+str(data.index[t]))
        print('')
        if signal == 0:
            print('no trading')
        elif  signal == 1:
            print('long position on '+stock2+' and short position on '+stock1)
        else:
            print('long position on '+stock1+' and short position on '+stock2)
        print('gross daily return: '+str(round(gross_return*100,2))+'%')
        print('net daily return: '+str(round(net_return*100,2))+'%')
        print('cumulative net return so far: '+str(round(np.prod(1+net_returns)*100-100,2))+'%')
        print('')
        old_signal = signal
    #plotting equity curves
    plt.plot(np.append(1,np.cumprod(1+gross_returns)))
    plt.plot(np.append(1,np.cumprod(1+net_returns)))


day 2020-12-29 00:00:00

no trading
gross daily return: 0%
net daily return: 0.0%
cumulative net return so far: 0.0%

day 2020-12-30 00:00:00

no trading
gross daily return: 0%
net daily return: 0.0%
cumulative net return so far: 0.0%

day 2020-12-31 00:00:00

long position on AAPL and short position on JPM
gross daily return: -1.53%
net daily return: -1.63%
cumulative net return so far: -1.63%

day 2021-01-04 00:00:00

no trading
gross daily return: 0%
net daily return: -0.1%
cumulative net return so far: -1.73%

day 2021-01-05 00:00:00

no trading
gross daily return: 0%
net daily return: 0.0%
cumulative net return so far: -1.73%

day 2021-01-06 00:00:00

no trading
gross daily return: 0%
net daily return: 0.0%
cumulative net return so far: -1.73%

day 2021-01-07 00:00:00

no trading
gross daily return: 0%
net daily return: 0.0%
cumulative net return so far: -1.73%

day 2021-01-08 00:00:00

no trading
gross daily return: 0%
net daily return: 0.0%
cumulative net return so far: -1.73%



KeyboardInterrupt: 